# Managing Kleio files

> First time use: follow instructions in the [README.md](README.md) file in this directory.

## Initial setup

Create a TimelinkNotebook object. This will be the interface for most of your interactions with Timelink.

The first time you run this notebook, you will be prompted to install ìpykernel. This is required to run Timelink in a notebook.

Note:
* First time run takes a little time as the required Docker images are downloaded.
* Timelink will default to using sqlite as the database, see [Receipts notebook for more control](1-receipts.ipynb)


In [13]:
from timelink.notebooks import TimelinkNotebook

tlnb = TimelinkNotebook()
tlnb.print_info()


INFO:root:Database is healthy
2025-05-23 14:59:57,660 - INFO - Database is healthy
INFO:alembic.runtime.migration:Context impl SQLiteImpl.
2025-05-23 14:59:57,660 - INFO - Database is healthy
INFO:alembic.runtime.migration:Context impl SQLiteImpl.
2025-05-23 14:59:57,664 - INFO - Context impl SQLiteImpl.
INFO:alembic.runtime.migration:Will assume non-transactional DDL.
2025-05-23 14:59:57,664 - INFO - Will assume non-transactional DDL.
2025-05-23 14:59:57,664 - INFO - Context impl SQLiteImpl.
INFO:alembic.runtime.migration:Will assume non-transactional DDL.
2025-05-23 14:59:57,664 - INFO - Will assume non-transactional DDL.
2025-05-23 14:59:57,793 - WARNING - Reusing existing container jolly_noyce
INFO:root:Kleio server started successfully
2025-05-23 14:59:57,801 - INFO - Kleio server started successfully
2025-05-23 14:59:57,793 - WARNING - Reusing existing container jolly_noyce
INFO:root:Kleio server started successfully
2025-05-23 14:59:57,801 - INFO - Kleio server started successfu

Timelink version: 1.1.25
Project name: dehergne-repertoire
Project home: /Users/jrc/develop/dehergne-repertoire
Database type: sqlite
Database name: dehergne_repertoire
Kleio image: timelinkserver/kleio-server
Kleio server token: OPlFi...
Kleio server URL: http://127.0.0.1:8088
Kleio server home: /Users/jrc/develop/dehergne-repertoire
Kleio server container: jolly_noyce
Kleio version requested: latest
Kleio server version: 12.8.593 (2025-03-16 21:55:53)
SQLite directory: /Users/jrc/develop/dehergne-repertoire/database/sqlite
Database version: 6ccf1ef385a6
Call print_info(show_token=True) to show the Kleio Server token
Call print_info(show_password=True) to show the Postgres password
TimelinkNotebook(project_name=dehergne-repertoire, project_home=/Users/jrc/develop/dehergne-repertoire, db_type=sqlite, db_name=dehergne_repertoire, kleio_image=timelinkserver/kleio-server, kleio_version=latest, postgres_image=postgres, postgres_version=latest)


TimelinkNotebook created a database to store data and a Kleio server to 
translate transcriptions in Kleio notation into data that can be imported
into a database.

To get information about the database and the Kleio server do ```tnlb.print_info()```

### Database status

Count the number of rows in each table in the database.


In [14]:
tlnb.table_row_count_df()

,table,count
0,acts,28
1,alembic_version,1
2,aregisters,0
3,attributes,26742
4,blinks,67
5,class_attributes,58
6,classes,10
7,entities,32878
8,geoentities,359
9,kleiofiles,29


### Show the kleio files available 

In [15]:
kleio_files = tlnb.get_kleio_files()
kleio_files[['name','status','errors','warnings','import_status','import_status','import_errors']]


,name,status,errors,warnings,import_status,import_status,import_errors
0,dehergne-0-abrev.cli,V,0,0,I,I,0
1,dehergne-a.cli,V,0,0,I,I,0
2,dehergne-b.cli,V,0,0,I,I,0
3,dehergne-c.cli,V,0,0,I,I,0
4,dehergne-d.cli,V,0,0,I,I,0
5,dehergne-e.cli,V,0,0,I,I,0
6,dehergne-f.cli,T,0,0,I,I,0
7,dehergne-g.cli,V,0,0,I,I,0
8,dehergne-h.cli,V,0,0,I,I,0
9,dehergne-i.cli,V,0,0,I,I,0


## Optionally clean translations 

Do this if there was an update to the Kleio server and you want to make sure that all translations are redone to incorporate the changes.

In [ ]:
# NBVAL_IGNORE_OUTPUT

# tlnb.kleio_server.translation_clean("", recurse="yes")


/Users/jrc/mhk-home/sources/dehergne/.venv/lib/python3.10/site-packages/timelink/api/models/pom_som_mapper.py:698: UserWarning: Ambiguous mapping! more than one element of rel is mapped to column class date check mapping for this group
  element: KElement = group.get_element_by_name_or_class(cattr.colclass)


### Translate and import as needed

In [17]:

tlnb.db.update_from_sources(path='', with_import_errors=True)

INFO:root:Request translation of T sources/dehergne-p.cli
2025-05-23 15:04:36,821 - INFO - Request translation of T sources/dehergne-p.cli
2025-05-23 15:04:36,821 - INFO - Request translation of T sources/dehergne-p.cli
INFO:root:Importing sources/dehergne-p.cli
2025-05-23 15:04:46,252 - INFO - Importing sources/dehergne-p.cli
INFO:root:Importing sources/dehergne-p.cli
2025-05-23 15:04:46,252 - INFO - Importing sources/dehergne-p.cli
INFO:root:Storing 30 postponed relations
2025-05-23 15:04:51,233 - INFO - Storing 30 postponed relations
INFO:root:Storing 30 postponed relations
2025-05-23 15:04:51,233 - INFO - Storing 30 postponed relations


## Check the status of the files

Check the import status of the translated files

* I: Imported
* E: Imported with error
* W: Imported with warnings no errors
* N: Not imported
* U: Translation updated need to reimport

### Check a translation report

If needed to check for errors.


In [8]:

# get the import status
imported_files_df = tlnb.get_import_status()
errors = imported_files_df[(imported_files_df["errors"] > 0) | (imported_files_df["import_errors"] > 0)]
errors[["name", "errors", "warnings", "import_status", "import_errors", "import_warnings", "path"]].sort_values("name")

,name,errors,warnings,import_status,import_errors,import_warnings,path


In [9]:
file_number=list(errors.index.unique())
if len(file_number) > 0:
    rpt = tlnb.get_translation_report(imported_files_df, file_number)
    print(rpt)
else:
    print("No errors found")



No errors found


### Get the import report for a file

In case there are errors in the import phase check the import report.

In [10]:
ierrors = imported_files_df[imported_files_df.import_errors>0][["import_status", "import_errors","import_warnings","name","imported","errors","warnings","path"]].sort_values("name")
ierrors

,import_status,import_errors,import_warnings,name,imported,errors,warnings,path


In [11]:

rpt = tlnb.get_import_rpt(imported_files_df, rows=list(ierrors.index), match_path=True)
print(rpt)

In [ ]:
tlnb.get_import_rpt('dehergne-a.cli')  # just the file name of the kleio file

'No errors\nNo warnings'


### Keept the database update as Kleio files are translated
Updates source translations and imports into database sources with no errors.

Only changed files since last import, or new files, will be processed.

In [ ]:
import logging
import os
import time
logging.basicConfig(level=logging.INFO)
# show logging in the notebook
from IPython.display import display, HTML
display(HTML("<style>.jp-OutputArea-output { font-size: 1.5em; }</style>"))
# Set up logging to display in the notebook
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
handler.setLevel(logging.INFO)
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
logger.addHandler(handler)
# Set up the logging configuration
# Set the logging level to INFO
# Set the format for the log messages
kleio_home_path = tlnb.kleio_server.get_kleio_home()
print(f"kleio_home_path: {kleio_home_path}")
# Monitor the directory for changes
def monitor_kleio_home(path):
    logging.info(f"Monitoring changes in {path}")
    previous_files = {f: os.path.getmtime(os.path.join(path, f)) for f in os.listdir(path)}
    while True:
        time.sleep(10)  # Check every 10 seconds
        current_files = {f: os.path.getmtime(os.path.join(path, f)) for f in os.listdir(path)}
        changed_files = [f for f in current_files if f not in previous_files or current_files[f] != previous_files[f]]
        if changed_files:
            logging.info(f"Detected changes in files: {changed_files}")
            tlnb.update_from_sources(path='')
        previous_files = current_files
tlnb.db.update_from_sources(path='', with_import_errors=True)
monitor_kleio_home(kleio_home_path)